In [1]:
import pandas as pd
import tensorflow as tf

from pathlib import Path
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

pd.set_option('display.max_columns', None)

In [2]:
df = pd.read_csv(Path('..', 'data', 'alzheimers_disease_data.csv'))

In [3]:
# Drop unnecessary columns
df.drop(columns=['PatientID', 'DoctorInCharge'], inplace=True)

In [4]:
# Split preprocessed data into features and target arrays
X = df.drop(columns='Diagnosis')
y = df['Diagnosis']

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y)

# Scale the data
scaler = StandardScaler()
X_scaler = scaler.fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

# Determine the number of input neurons
print(f'Input neurons: {len(X_train_scaled[0])}')

Input neurons: 32


In [5]:
# Define the model
nn = tf.keras.models.Sequential()
nn.add(tf.keras.layers.Input(shape=(32,)))
nn.add(tf.keras.layers.Dense(units=80, activation='relu'))
nn.add(tf.keras.layers.Dense(units=1, activation='sigmoid'))

# Check the structure of the model
nn.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 80)             │         2,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,721 (10.63 KB)

 Trainable params: 2,721 (10.63 KB)

 Non-trainable params: 0 (0.00 B)

In [6]:
# Compile the model
nn.compile(loss="binary_crossentropy", optimizer="adam", metrics=["accuracy"])

results = []
epochs_trained = 0

for i in range(20):
    # Train the model
    trained_model = nn.fit(
        X_train_scaled, 
        y_train, 
        epochs=5
    )

    epochs_trained += len(trained_model.epoch)

    # Evaluate the model using the test data
    model_loss, model_accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
    results.append({
        'Epochs Trained' : epochs_trained,
        'Accuracy' : model_accuracy,
        'Loss' : model_loss
    })

results_df = pd.DataFrame(results)
results_df.set_index('Epochs Trained', inplace=True)
results_df['Acurracy - Loss'] = results_df['Accuracy'] - results_df['Loss']
results_df.sort_values('Acurracy - Loss', ascending=False, inplace=True)

Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5777 - loss: 0.7015
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.7736 - loss: 0.5124
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8342 - loss: 0.4362
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8491 - loss: 0.3796
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8496 - loss: 0.3603
17/17 - 0s - 11ms/step - accuracy: 0.8401 - loss: 0.4073
Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8605 - loss: 0.3614
Epoch 2/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8561 - loss: 0.3509
Epoch 3/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8742 - loss: 0.3136
Epoch 4/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8738 - loss: 0.3022
Epoch 5/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8766 - loss: 0.3153
17/17 - 0s - 3ms/step - accuracy: 0.8476 - loss: 0.3978
Epoch 1/5
51/51 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accura

In [7]:
print('Maximum accuracy: {} at {} epochs'.format(round(results_df['Accuracy'].max(), 4), results_df['Accuracy'].idxmax()))
print('Minimum loss:     {} at {} epochs'.format(round(results_df['Loss'].min(), 4), results_df['Loss'].idxmin()))
results_df

Maximum accuracy: 0.8494 at 20 epochs
Minimum loss:     0.3927 at 20 epochs


,Accuracy,Loss,Acurracy - Loss
Epochs Trained,,,
20,0.849442,0.392740,0.456702
15,0.849442,0.394119,0.455323
25,0.845725,0.393810,0.451915
10,0.847584,0.397825,0.449759
30,0.845725,0.402583,0.443142
35,0.849442,0.416316,0.433126
5,0.840149,0.407338,0.432811
40,0.840149,0.428363,0.411786
45,0.836431,0.445808,0.390623
